In [4]:
import numpy as np
import os
import pickle
from karabo.imaging.imager_oskar import OskarDirtyImager
from karabo.imaging import imager_wsclean
from karabo.simulation.interferometer import InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulation.visibility import Visibility
from karabo.imaging.imager_oskar import OskarDirtyImager, OskarDirtyImagerConfig
from karabo.simulator_backend import SimulatorBackend
from astropy.io import fits
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.use('Tkagg')
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    return

In [5]:
def get_baselines(telescope_path):
    print(telescope_path)
    layout=np.loadtxt(telescope_path+'/layout.txt')
    nant=len(layout)
    nb=int(nant*(nant-1)*0.5)
    base_length=[0]*nb
    k=0
    for i in range(nant):
        for j in range(i,nant):
            if(i!=j):
                base_length[k] = np.sqrt((layout[i][0]-layout[j][0])**2 + (layout[i][1]-layout[j][1])**2)
                k=k+1
    base_length=np.array(base_length)
    telname = telescope_path.split('/')[-1].split('.')[0]
    return base_length,nant,nb,telname

In [ ]:
meerkat_path = '/data/rohit/skao_repo/meerkat.tm'
ska1mid_path = '/data/rohit/skao_repo/ska1mid.tm'
skamid_AAs_path = '/data/rohit/skao_repo/ska-mid-AAstar.tm'
skamid_AA2_path = '/data/rohit/skao_repo/ska-mid-AA2.tm'
skamid_AA1_path = '/data/rohit/skao_repo/ska-mid-AA1.tm'
skamid_AA0_path = '/data/rohit/skao_repo/ska-mid-AA0.5.tm'
#---------------
mwa_ph1_path = '/data/rohit/skao_repo/mwa.phase1.tm'
ska1low_path = '/data/rohit/skao_repo/ska1low.tm'
skalow_AAs_path = '/data/rohit/skao_repo/ska-low-AAstar.tm'
skalow_AA2_path = '/data/rohit/skao_repo/ska-low-AA2.tm'
skalow_AA1_path = '/data/rohit/skao_repo/ska-low-AA1.tm'
skalow_AA0_path = '/data/rohit/skao_repo/ska-low-AA0.5.tm'

telescope_mid_list = [ska1mid_path, skamid_AAs_path, skamid_AA2_path, skamid_AA1_path,skamid_AA0_path, meerkat_path]
telescope_low_list = [ska1low_path, skalow_AAs_path, skalow_AA2_path, skalow_AA1_path,skalow_AA0_path, mwa_ph1_path]
telescope_list = telescope_low_list + telescope_mid_list

freq_low_arr = np.arange(50,350) # in MHz
freq_mid_arr = np.arange(350,15000) # in MHz
band_start=np.array([50,350,950,1650,2800,4600,8300])
band_end=np.array([350,1050,1760,3050,5180,8500,15400])
band_mid = 0.5*(band_start+band_end)

base_length = [0]*len(telescope_list)
nant = [0]*len(telescope_list)
nb = [0]*len(telescope_list)
nbmax = [0]*len(telescope_list)
nbmedian = [0]*len(telescope_list)
telname = [0]*len(telescope_list)
beamsize_arr = [0]*len(telescope_list)
beamsize_arr_arcsec = [0]*len(telescope_list)
beamsize_arr_median_arcsec = [0]*len(telescope_list)
i=0
for tel in telescope_list:
    base_length[i],nant[i],nb[i],telname[i] = get_baselines(telescope_list[i])
    nbmax[i] = base_length[i].max()
    nbmedian[i] = np.median(base_length[i])
    if(i<5):
        freq_list = freq_mid_arr
    else:
        freq_list = freq_low_arr
    #print(telname[i], freq_list)
    beamsize_arr[i] = 3.e8/base_length[i].max()/(band_mid*1.e6)
    beamsize_arr_arcsec[i] = beamsize_arr[i]*180/np.pi*3600
    beamsize_arr_arcsec[i] = 3.e8/np.median(base_length[i])/(band_mid*1.e6)*180/np.pi*3600
    i=i+1
#print('Maximum Baseline',base_length.max())
#----------------------------

/data/rohit/skao_repo/ska1low.tm
/data/rohit/skao_repo/ska-low-AAstar.tm
/data/rohit/skao_repo/ska-low-AA2.tm
/data/rohit/skao_repo/ska-low-AA1.tm
/data/rohit/skao_repo/ska-low-AA0.5.tm
/data/rohit/skao_repo/mwa.phase1.tm
/data/rohit/skao_repo/ska1mid.tm
/data/rohit/skao_repo/ska-mid-AAstar.tm
/data/rohit/skao_repo/ska-mid-AA2.tm
/data/rohit/skao_repo/ska-mid-AA1.tm
/data/rohit/skao_repo/ska-mid-AA0.5.tm
/data/rohit/skao_repo/meerkat.tm


In [24]:
for i in range(len(telescope_list)):
    print(telname[i],'|',nb[i],'|',np.round(nbmax[i]),'m|',np.round(nbmedian[i])),'m'

band_list = ['Band-low','Band-mid1','Band-mid2','Band-mid3','Band-mid4','Band-mid5a','Band-mid5b']
for i in range(len(band_list)):
    print(band_list[i],'&',
        np.round(beamsize_arr_arcsec[4][i],2),'&',  
        np.round(beamsize_arr_arcsec[10][i],2),'&',
        np.round(beamsize_arr_arcsec[3][i],2),'&',
        np.round(beamsize_arr_arcsec[9][i],2),'&',
        np.round(beamsize_arr_arcsec[2][i],2),'&',
        np.round(beamsize_arr_arcsec[8][i],2),'&',
        np.round(beamsize_arr_arcsec[1][i],2),'&',
        np.round(beamsize_arr_arcsec[7][i],2),'&',  
        np.round(beamsize_arr_arcsec[0][i],2),'&',
        np.round(beamsize_arr_arcsec[6][i],2),'&',
        np.round(beamsize_arr_arcsec[5][i],2),'&',
        np.round(beamsize_arr_arcsec[11][i],2),
        '\\\\'
        )



ska1low | 130816 | 65413.0 m| 4904.0
ska-low-AAstar | 46971 | 65369.0 m| 1229.0
ska-low-AA2 | 2016 | 61288.0 m| 3324.0
ska-low-AA1 | 120 | 44883.0 m| 3683.0
ska-low-AA0 | 6 | 5716.0 m| 4246.0
mwa | 8128 | 2874.0 m| 622.0
ska1mid | 19306 | 157397.0 m| 2838.0
ska-mid-AAstar | 10296 | 106550.0 m| 2223.0
ska-mid-AA2 | 2016 | 106550.0 m| 2124.0
ska-mid-AA1 | 28 | 1415.0 m| 665.0
ska-mid-AA0 | 6 | 1415.0 m| 1102.0
meerkat | 2016 | 7698.0 m| 957.0
Band-low & 54.13 & 218.59 & 6.89 & 218.59 & 5.05 & 2.9 & 4.73 & 2.9 & 4.73 & 1.97 & 107.67 & 40.19 \\
Band-mid1 & 15.47 & 62.45 & 1.97 & 62.45 & 1.44 & 0.83 & 1.35 & 0.83 & 1.35 & 0.56 & 30.76 & 11.48 \\
Band-mid2 & 7.99 & 32.26 & 1.02 & 32.26 & 0.75 & 0.43 & 0.7 & 0.43 & 0.7 & 0.29 & 15.89 & 5.93 \\
Band-mid3 & 4.61 & 18.6 & 0.59 & 18.6 & 0.43 & 0.25 & 0.4 & 0.25 & 0.4 & 0.17 & 9.16 & 3.42 \\
Band-mid4 & 2.71 & 10.96 & 0.35 & 10.96 & 0.25 & 0.15 & 0.24 & 0.15 & 0.24 & 0.1 & 5.4 & 2.01 \\
Band-mid5a & 1.65 & 6.67 & 0.21 & 6.67 & 0.15 & 0.09 & 0.14 &

In [13]:

f,ax=plt.subplots(1,1)
ax.hist(nb[0],bins=100)
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Max. Resolution (arcsec)')
ax.set_title('SKA-mid')
plt.show()

/tmp/ipykernel_2200842/1708583086.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
